# Notebook 1: Introduction with Shallow Water - Williamson 2 Test Case

This document will provide an introduction to Gusto by going through an example script of the Williamson 2 test case. We will demonstrate setting up the problem, solving it, and examining the results and diagnostics.

We are solving the shallow water equations in this example. We will use the following set of momentum and continuity equations, for a velocity $\textbf{u}$ and free surface, $\eta$:
$$\textbf{u}_t + f \textbf{u}^{\perp} + g \nabla \eta + (\textbf{u} \cdot \nabla) \textbf{u} = 0$$
$$\eta_t + H(\nabla \cdot \textbf{u}) + \nabla [\textbf{u} (\eta - b)] = 0$$

The parameters we need to specify will be the Coriolis force, $f$, gravitational constant, $g$, and mean depth, $H$.
$\newline$
We don't have to worry about boundary conditions, due to solving on a spherical domain. 

We begin by importing the required libraries and functions from Firedrake and Gusto:

In [1]:
from gusto import *
from firedrake import IcosahedralSphereMesh, SpatialCoordinate, as_vector
from math import pi
import sys

firedrake:WARNING OMP_NUM_THREADS is not set or is set to a value greater than 1, we suggest setting OMP_NUM_THREADS=1 to improve performance


We now define the time-step size and the duration of the simulation. We wish to analyse the solution five times over one day.

In [4]:
dt = 4000.
day = 24.*60.*60.
tmax = 1*day

ndumps = 5
dumpfreq = int(tmax / (ndumps*dt))

We next specify the parameters for our shallow water model. We are solving on the Earth, so we give the relevant radius and height.

In [5]:
# setup shallow water parameters
R = 6371220.
H = 5960.

# setup input that doesn't change with ref level or dt
parameters = ShallowWaterParameters(H=H)

Next, we define the mesh. We will choose an Icosahedral Sphere Mesh. This is formed using the desired refinement level.

In [6]:
#Set up the mesh and choose the refinement level
ref_level = 3  # number of horizontal cells = 20*(4^refinements)

mesh = IcosahedralSphereMesh(radius=R,
                             refinement_level=ref_level, degree=3)
x = SpatialCoordinate(mesh)
mesh.init_cell_orientations(x)

We need to specify an output directory name before running the code. We will record the test case, refinement level and time-step size in the name. To prevent overwriting data, Gusto will not enable overwriting an existing file. Hence, if one wishes to re-run a simulation, the existing results file needs to be removed first. 

In [7]:
output = OutputParameters(dirname="sw_W2_ref%s_dt%s" % (ref_level, dt),
                          dumpfreq=dumpfreq,
                          steady_state_error_fields=['u', 'D'],
                          log_level='INFO')

We can specify which diagnostics we wish to record over the simulation. We then pass these into the State function. This will initialise the problem with the relevant mesh, parameters, and time-step. This state is updated after each iteration of the chosen time-stepper.

In [8]:
diagnostic_fields = [RelativeVorticity(), PotentialVorticity(),
                         ShallowWaterKineticEnergy(),
                         ShallowWaterPotentialEnergy(),
                         ShallowWaterPotentialEnstrophy(),
                        CourantNumber()]

state = State(mesh,
              dt=dt,
              output=output,
              parameters=parameters,
              diagnostic_fields=diagnostic_fields)

gusto:INFO Physical parameters that take non-default values:
gusto:INFO H: 5960.0


A spatially varying Coriolis force is defined over the sphere. Due to using a co-ordinate system based on ..., we can apply $f = 2 \Omega $. We then pass this function into the set-up for the Shallow Water Equations.

In [9]:
#Create a spatially varying function for the Coriolis force:
Omega = parameters.Omega
x = SpatialCoordinate(mesh)
fexpr = 2*Omega*x[2]/R
eqns = ShallowWaterEquations(state, "BDM", 1, fexpr=fexpr)

Set up the initial conditions:
Due to the choice of function spaces for the velocity and depth, the intialisations are performed using projection 
and interpolation procedures respectively. Williamson 2 specifies initial conditions for a 'Global Steady State Non-linear Zonal Geostrophic Zone' ... . We only require a two-dimensional velocity, so will specify the third component to be zero. 

In [10]:
u0 = state.fields("u")
D0 = state.fields("D")
u_max = 2*pi*R/(12*day)  # Maximum amplitude of the zonal wind (m/s)
uexpr = as_vector([-u_max*x[1]/R, u_max*x[0]/R, 0.0])
g = parameters.g
Dexpr = H - ((R * Omega * u_max)*(x[2]*x[2]/(R*R)))/g
u0.project(uexpr)
D0.interpolate(Dexpr)

Coefficient(WithGeometry(IndexedProxyFunctionSpace(<firedrake.mesh.MeshTopology object at 0x7f25506068b0>, FiniteElement('Discontinuous Lagrange', triangle, 1, variant='equispaced'), name='DG1', index=1, component=None), Mesh(VectorElement(FiniteElement('Lagrange', Cell('triangle', 3), 3), dim=3), 4)), 26)

We next define the choice of time steppers. We will use the Crank-Nicolson approach, which means we can specify different approaches for solving the velocity and depth fields. We choose to use an Implicit Midpoint method for the velocity and an explicit strong stability preserving RK3 method for the depth. 

In [11]:
#Now, construct the time-stepper. We will firstly use a semi-implicit (?) approach.
#We will neglect any transport schemes for now.
transport_schemes = [ImplicitMidpoint(state, "u"),
                          SSPRK3(state, "D", subcycles=2)]
stepper = CrankNicolson(state, eqns, transport_schemes)

In [12]:
#Run the time-stepper and generate the output
stepper.run(t=0, tmax=tmax)

/home/timandrews/firedrake/src/firedrake/firedrake/checkpointing.py:89: DeprecationWarning: DumbCheckpoint class will be deprecated after 01/01/2023; use CheckpointFile class instead.
  warnings.warn("DumbCheckpoint class will be deprecated after 01/01/2023; use CheckpointFile class instead.",
gusto:INFO at start of timestep, t=0, dt=4000.0
gusto:INFO at start of timestep, t=4000.0, dt=4000.0
gusto:INFO at start of timestep, t=8000.0, dt=4000.0
gusto:INFO at start of timestep, t=12000.0, dt=4000.0
gusto:INFO at start of timestep, t=16000.0, dt=4000.0
gusto:INFO at start of timestep, t=20000.0, dt=4000.0
gusto:INFO at start of timestep, t=24000.0, dt=4000.0
gusto:INFO at start of timestep, t=28000.0, dt=4000.0
gusto:INFO at start of timestep, t=32000.0, dt=4000.0
gusto:INFO at start of timestep, t=36000.0, dt=4000.0
gusto:INFO at start of timestep, t=40000.0, dt=4000.0
gusto:INFO at start of timestep, t=44000.0, dt=4000.0
gusto:INFO at start of timestep, t=48000.0, dt=4000.0
gusto:INFO 

Now, you should be able to find the relevant file in the results folder created in this directory. The output at the specified times is saved in a format that can be viewed using Paraview. This can be seen as five .vtu files (field_output1.vtu, field_output2.vtu, etc.), as well as the original one for the initial conditions (field_output0.vtu). Documentation on how to use this is given here:  . Here is what you would get if you ran the simulation for five days:

Congratulations, you have now successfully run a Gusto script!